##### importando bibliotecas

In [1]:
# importando bibliotecas
import pandas as pd
from urllib.request import Request, urlopen, urlretrieve
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import os

##### primeiro teste

In [2]:
# primeiros testes
url = 'https://alura-site-scraping.herokuapp.com/hello-world.php'
response = urlopen(url)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')

print(soup.find('h1', {'class':'sub-header'}).get_text())
print(soup.find('h1', id='hello-world').get_text())
print(soup.find('p').get_text())

Curso de Web Scraping
Hello World!!!
Web Scraping é o termo utilizado para definir a prática de coletar automaticamente informações na Internet. Isto é feito, geralmente, por meio de programas que simulam a navegação humana na Web.


##### importando página 'alura motors'

In [3]:
# coletando dados de 'alura motors'
url = 'https://alura-site-scraping.herokuapp.com/index.php'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

try:
    req = Request(url, headers=headers)
    response = urlopen(req)
except HTTPError as e:
    print(e.status, e.reason)
except URLError as e:
    print(e.reason)

html = response.read()          # type(html) => bytes
html = html.decode('utf-8')     # type(html) => str

# transforma o html em uma lista, sem caracteres especiais; une os itens da lista com espaço ' '; elimina espaços em branco entre as tags
def trata_html(input):
    return ' '.join(input.split()).replace('> <', '><')

html = trata_html(html)
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <title>
   Alura Motors
  </title>
  <style>
   /*Regra para a animacao*/ @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } } /*Mudando o tamanho do icone de resposta*/ div.glyphicon { color:#6B8E23; font-size: 38px; } /*Classe que mostra a animacao 'spin'*/ .loader { border: 16px solid #f3f3f3; border-radius: 50%; border-top: 16px solid #3498db; width: 80px; height: 80px; -webkit-animation: spin 2s linear infinite; animation: spin 2s linear infinite; }
  </style>
  <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" rel="stylesheet"/>
  <link href="css/styles.css" media="all" rel="stylesheet"/>
  <script src="https://code.jquery.com/jquery-1.12.4.js">
  

##### coletando dados do primeiro card de anúncio

In [4]:
# criando um dicionário vazio
card = {}

In [5]:
# buscando a primeira <div> com class='well card' para coletar informações do primeiro card de anúncio
anuncio = soup.find('div', {'class':'well card'})
anuncio

<div class="well card"><div class="col-md-3 image-card"><img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/></div><div class="col-md-6 body-card"><p class="txt-name inline">LAMBORGHINI AVENTADOR</p><p class="txt-category badge badge-secondary inline">USADO</p><p class="txt-motor">Motor 1.8 16v</p><p class="txt-description">Ano 1993 - 55.286 km</p><ul class="lst-items"><li class="txt-items">► 4 X 4</li><li class="txt-items">► Câmera de estacionamento</li><li class="txt-items">► Controle de tração</li><li class="txt-items">► Sensor de estacionamento</li><li class="txt-items">...</li></ul><p class="txt-location">Belo Horizonte - MG</p></div><div class="col-md-3 value-card"><div class="value"><p class="txt-value">R$ 338.000</p></div></div></div>

In [6]:
# buscando o conteúdo (valor) do primeiro <p> com class='txt-value'
anuncio.find('p', {'class':'txt-value'}).get_text()

'R$ 338.000'

In [7]:
# adicionando o valor encontrado ao dicionário 'card'
card['value'] = anuncio.find('p', {'class':'txt-value'}).get_text()
card

{'value': 'R$ 338.000'}

In [8]:
# buscando o conteúdo (informações) da <div> com a class='body-card' e adicionando a variável 'info'
infos = anuncio.find('div', {'class':'body-card'}).find_all('p')
infos

[<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>,
 <p class="txt-category badge badge-secondary inline">USADO</p>,
 <p class="txt-motor">Motor 1.8 16v</p>,
 <p class="txt-description">Ano 1993 - 55.286 km</p>,
 <p class="txt-location">Belo Horizonte - MG</p>]

In [9]:
# selecionando o nome de cada 'class' de 'infos' para usar como chave do dicionário
for info in infos:
    print(info.get('class')[0])

txt-name
txt-category
txt-motor
txt-description
txt-location


In [10]:
# usando separador '-' para separar 'txt'
for info in infos:
    print(info.get('class')[0].split('-'))

['txt', 'name']
['txt', 'category']
['txt', 'motor']
['txt', 'description']
['txt', 'location']


In [11]:
# coletando o último item para usar como chave do dicionário
for info in infos:
    print(info.get('class')[0].split('-')[-1])

name
category
motor
description
location


In [12]:
# coletando os valores de 'info'
for info in infos:
    print(info.get_text())

LAMBORGHINI AVENTADOR
USADO
Motor 1.8 16v
Ano 1993 - 55.286 km
Belo Horizonte - MG


In [13]:
# adicionando chaves e valores de 'info' a variável 'card'
for info in infos:
    card[info.get('class')[0].split('-')[-1]] = info.get_text()
card

{'value': 'R$ 338.000',
 'name': 'LAMBORGHINI AVENTADOR',
 'category': 'USADO',
 'motor': 'Motor 1.8 16v',
 'description': 'Ano 1993 - 55.286 km',
 'location': 'Belo Horizonte - MG'}

In [14]:
# buscando o conteúdo (items) da <div> pela class='body-card' e pelos itens <li> da lista <ul>, e adicionando a variável 'items'
items = anuncio.find('div', {'class':'body-card'}).ul.find_all('li')
items

[<li class="txt-items">► 4 X 4</li>,
 <li class="txt-items">► Câmera de estacionamento</li>,
 <li class="txt-items">► Controle de tração</li>,
 <li class="txt-items">► Sensor de estacionamento</li>,
 <li class="txt-items">...</li>]

In [15]:
# removendo o último item '...'
items.pop()

<li class="txt-items">...</li>

In [16]:
# coletando o valor de cada item
for item in items:
    print(item.getText())

► 4 X 4
► Câmera de estacionamento
► Controle de tração
► Sensor de estacionamento


In [17]:
# tratando o valor de cada item
for item in items:
    print(item.getText().replace('► ', ''))

4 X 4
Câmera de estacionamento
Controle de tração
Sensor de estacionamento


In [18]:
# coletando o valor de cada item, tratando e colocando na lista 'acessorios'
acessorios = []
for item in items:
    acessorios.append(item.getText().replace('► ',''))
acessorios

['4 X 4',
 'Câmera de estacionamento',
 'Controle de tração',
 'Sensor de estacionamento']

In [19]:
# adicionando a chave 'items' e os valores da lista 'acessorios' ao dicionário 'card'
card['items'] = acessorios
card

{'value': 'R$ 338.000',
 'name': 'LAMBORGHINI AVENTADOR',
 'category': 'USADO',
 'motor': 'Motor 1.8 16v',
 'description': 'Ano 1993 - 55.286 km',
 'location': 'Belo Horizonte - MG',
 'items': ['4 X 4',
  'Câmera de estacionamento',
  'Controle de tração',
  'Sensor de estacionamento']}

In [20]:
# criando um DF a partir do dicionário 'card'
dataset = pd.DataFrame.from_dict(card)
dataset

,value,name,category,motor,description,location,items
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,4 X 4
1,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,Câmera de estacionamento
2,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,Controle de tração
3,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,Sensor de estacionamento


In [21]:
# criando DF novamente, orientando pelo 'index'
dataset = pd.DataFrame.from_dict(card, orient='index')
dataset

,0
value,R$ 338.000
name,LAMBORGHINI AVENTADOR
category,USADO
motor,Motor 1.8 16v
description,Ano 1993 - 55.286 km
location,Belo Horizonte - MG
items,"[4 X 4, Câmera de estacionamento, Controle de ..."


In [22]:
# criando DF novamente, usando o 'transpose' para trocar linhas e colunas
dataset = pd.DataFrame.from_dict(card, orient='index').T
dataset

,value,name,category,motor,description,location,items
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"[4 X 4, Câmera de estacionamento, Controle de ..."


In [23]:
# exportando o DF para 'csv'
dataset.to_csv('./output/data/dataset.csv', sep=';', index=False, encoding='utf-8-sig')

In [24]:
# coletando link da imagem usando a class='image-card'
image = anuncio.find('div', {'class':'image-card'}).img
image.get('src')

'https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg'

In [25]:
# mostrando a imagem coletada
display(HTML('<img src=' + image.get('src') + '>'))

In [26]:
# salvando a imagem coletada em 'png'
nome_imagem = image.get('src').split('/')[-1].replace('jpg', 'png')
urlretrieve(image.get('src'), './output/img/' + nome_imagem)

('./output/img/lamborghini-aventador-2932196__340.png',
 <http.client.HTTPMessage at 0x21caa70b580>)

##### coletando todos os anúncios da primeira página

In [27]:
# buscando todas as <div> com class='card' dentro da <div> com if='container-cards' e imprimindo pulando duas linhas
anuncios = soup.find('div', {'id': 'container-cards'}).find_all('div', class_='card')
for anuncio in anuncios:
    print(str(anuncio) + '\n\n')

<div class="well card"><div class="col-md-3 image-card"><img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/></div><div class="col-md-6 body-card"><p class="txt-name inline">LAMBORGHINI AVENTADOR</p><p class="txt-category badge badge-secondary inline">USADO</p><p class="txt-motor">Motor 1.8 16v</p><p class="txt-description">Ano 1993 - 55.286 km</p><ul class="lst-items"><li class="txt-items">► 4 X 4</li><li class="txt-items">► Câmera de estacionamento</li><li class="txt-items">► Controle de tração</li><li class="txt-items">► Sensor de estacionamento</li><li class="txt-items">...</li></ul><p class="txt-location">Belo Horizonte - MG</p></div><div class="col-md-3 value-card"><div class="value"><p class="txt-value">R$ 338.000</p></div></div></div>


<div class="well card"><div class="col-md-3 image-card"><img alt="Foto" height="155" src="https://caelum-online-publi

In [28]:
### script ###

# criando uma lista vazia
cards = []

# buscando todas as <div> com class='card' dentro da <div> com if='container-cards' e adicionando a variável 'anuncios'
anuncios = soup.find('div', {'id': 'container-cards'}).find_all('div', class_='card')

# coletando as informações dos cards
for anuncio in anuncios:
    card = {}

    # valor
    card['value'] = anuncio.find('p', {'class': 'txt-value'}).get_text()

    # informações
    infos = anuncio.find('div', {'class': 'body-card'}).find_all('p')
    for info in infos:
        card[info.get('class')[0].split('-')[-1]] = info.get_text()

    # acessórios
    items = anuncio.find('div', {'class': 'body-card'}).ul.find_all('li')
    items.pop()
    acessorios = []
    for item in items:
        acessorios.append(item.get_text().replace('► ', ''))
    card['items'] = acessorios

    # imagens
    image = anuncio.find('div', {'class': 'image-card'}).img
    nome_imagem = image.get('src').split('/')[-1].replace('jpg', 'png')
    if not os.path.isfile(f'./output/img/{nome_imagem}'):
        urlretrieve(image.get('src'), f'./output/img/{nome_imagem}')
    card['imagem'] = nome_imagem

    # adicionando resultado a lista 'cards'
    cards.append(card)

# criando um DF com os resultados
dataset = pd.DataFrame(cards)
# exportando o DF para 'csv'
dataset.to_csv('./output/data/dataset.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

,value,name,category,motor,description,location,items,imagem,opportunity
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"[4 X 4, Câmera de estacionamento, Controle de ...",lamborghini-aventador-2932196__340.png,NaN
1,R$ 346.000,BMW M2,USADO,Motor 3.0 32v,Ano 2018 - 83.447 km,Belo Horizonte - MG,"[Câmera de estacionamento, Controle de estabil...",bmw-m2-2970882__340.png,NaN
2,R$ 480.000,ALFA,USADO,Motor 1.8 16v,Ano 2004 - 19.722 km,Rio de Janeiro - RJ,"[Central multimídia, Bancos de couro, Rodas de...",alfa-1823056__340.png,NaN
3,R$ 133.000,PUECH,USADO,Motor Diesel V8,Ano 1992 - 34.335 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Rodas de liga, C...",puech-4055386__340.png,NaN
4,R$ 175.000,LAMBORGHINI MURCIELAGO,USADO,Motor 1.0 8v,Ano 1991 - 464 km,Belo Horizonte - MG,"[Central multimídia, Teto panorâmico, Sensor c...",lamborghini-murcielago-2872974__340.png,NaN
5,R$ 239.000,ASTON MARTIN,USADO,Motor Diesel V6,Ano 2004 - 50.189 km,Belo Horizonte - MG,"[Painel digital, Controle de tração, Teto pano...",aston-martin-2977916__340.png,OPORTUNIDADE
6,R$ 115.000,TVR,USADO,Motor 4.0 Turbo,Ano 2014 - 17.778 km,Belo Horizonte - MG,"[4 X 4, Teto panorâmico, Central multimídia, C...",tvr-2943925__340.png,NaN
7,R$ 114.000,EXCALIBUR,USADO,Motor 3.0 32v,Ano 2009 - 81.251 km,Rio de Janeiro - RJ,"[Painel digital, Câmbio automático, Sensor de ...",excalibur-2916730__340.png,NaN
8,R$ 75.000,MCLAREN,NOVO,Motor Diesel,Ano 2019 - 0 km,São Paulo - SP,"[Central multimídia, Câmera de estacionamento,...",mclaren-2855240__340.png,NaN
9,R$ 117.000,TOYOTA,USADO,Motor 4.0 Turbo,Ano 1999 - 12.536 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Piloto automátic...",toyota-3245397__340.png,OPORTUNIDADE


##### coletando todos os anúncios do site

In [29]:
# coletando a informação da quantidade de páginas, da tag <span> com class='info-pages'
soup.find('span', class_='info-pages').get_text()

'Página 1 de 25'

In [30]:
# separando os itens da string, coletando o último item e transformando em 'int'
paginas = int(soup.find('span', class_='info-pages').get_text().split()[-1])
paginas

25

In [31]:
# usando a quantidade de páginas para alterar a url
url = 'https://alura-site-scraping.herokuapp.com/index.php?page='
for i in range(paginas):
    print(url + str(i + 1))

https://alura-site-scraping.herokuapp.com/index.php?page=1
https://alura-site-scraping.herokuapp.com/index.php?page=2
https://alura-site-scraping.herokuapp.com/index.php?page=3
https://alura-site-scraping.herokuapp.com/index.php?page=4
https://alura-site-scraping.herokuapp.com/index.php?page=5
https://alura-site-scraping.herokuapp.com/index.php?page=6
https://alura-site-scraping.herokuapp.com/index.php?page=7
https://alura-site-scraping.herokuapp.com/index.php?page=8
https://alura-site-scraping.herokuapp.com/index.php?page=9
https://alura-site-scraping.herokuapp.com/index.php?page=10
https://alura-site-scraping.herokuapp.com/index.php?page=11
https://alura-site-scraping.herokuapp.com/index.php?page=12
https://alura-site-scraping.herokuapp.com/index.php?page=13
https://alura-site-scraping.herokuapp.com/index.php?page=14
https://alura-site-scraping.herokuapp.com/index.php?page=15
https://alura-site-scraping.herokuapp.com/index.php?page=16
https://alura-site-scraping.herokuapp.com/index.p

In [32]:
# criando uma lista vazia
cards = []

# transforma o html em uma lista, sem caracteres especiais; une os itens da lista com espaço ' '; elimina espaços em branco entre as tags
def trata_html(input):
    return ' '.join(input.split()).replace('> <', '><')

# obtendo o total de páginas
pages = int(soup.find('span', class_='info-pages').get_text().split()[-1])

# iterando por todas as páginas
for i in range(pages):

    # obtendo o HTML de cada página
    url = 'https://alura-site-scraping.herokuapp.com/index.php?page=' + str(i + 1)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
    try:
        req = Request(url, headers=headers)
        response = urlopen(req)
    except HTTPError as e:
        print(e.status, e.reason)
    except URLError as e:
        print(e.reason)
    html = response.read()          # type(html) => bytes
    html = html.decode('utf-8')     # type(html) => str
    html = trata_html(html)
    soup = BeautifulSoup(html, 'html.parser')

    # buscando todas as <div> com class='card' dentro da <div> com if='container-cards' e adicionando a variável 'anuncios'
    anuncios = soup.find('div', {'id':'container-cards'}).find_all('div', class_='card')

    # coletando as informações dos cards
    for anuncio in anuncios:
        card = {}

        # valor
        card['value'] = anuncio.find('p', {'class':'txt-value'}).getText()

        # informações
        infos = anuncio.find('div', {'class':'body-card'}).find_all('p')
        for info in infos:
            card[info.get('class')[0].split('-')[-1]] = info.get_text()

        # acessórios
        items = anuncio.find('div', {'class':'body-card'}).ul.find_all('li')
        items.pop()
        acessorios = []
        for item in items:
            acessorios.append(item.get_text().replace('► ', ''))
        card['items'] = acessorios

        # imagens
        image = anuncio.find('div', {'class':'image-card'}).img
        nome_imagem = image.get('src').split('/')[-1].replace('jpg', 'png')
        if not os.path.isfile(f'./output/img/{nome_imagem}'):
            urlretrieve(image.get('src'), f'./output/img/{nome_imagem}')
        card['image'] = nome_imagem

        # adicionando resultado a lista 'cards'
        cards.append(card)

# criando um DF com os resultados
dataset = pd.DataFrame(cards)
# exportando o DF para 'csv'
dataset.to_csv('./output/data/dataset.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

,value,name,category,motor,description,location,items,image,opportunity
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"[4 X 4, Câmera de estacionamento, Controle de ...",lamborghini-aventador-2932196__340.png,NaN
1,R$ 346.000,BMW M2,USADO,Motor 3.0 32v,Ano 2018 - 83.447 km,Belo Horizonte - MG,"[Câmera de estacionamento, Controle de estabil...",bmw-m2-2970882__340.png,NaN
2,R$ 480.000,ALFA,USADO,Motor 1.8 16v,Ano 2004 - 19.722 km,Rio de Janeiro - RJ,"[Central multimídia, Bancos de couro, Rodas de...",alfa-1823056__340.png,NaN
3,R$ 133.000,PUECH,USADO,Motor Diesel V8,Ano 1992 - 34.335 km,São Paulo - SP,"[Bancos de couro, Freios ABS, Rodas de liga, C...",puech-4055386__340.png,NaN
4,R$ 175.000,LAMBORGHINI MURCIELAGO,USADO,Motor 1.0 8v,Ano 1991 - 464 km,Belo Horizonte - MG,"[Central multimídia, Teto panorâmico, Sensor c...",lamborghini-murcielago-2872974__340.png,NaN
...,...,...,...,...,...,...,...,...,...
241,R$ 489.000,SUV REAR TIRE,USADO,Motor 3.0 32v,Ano 1998 - 74.292 km,São Paulo - SP,"[Câmera de estacionamento, Rodas de liga, Sens...",suv-rear-tire-2773267__340.png,NaN
242,R$ 427.000,ANTIQUE,NOVO,Motor 2.0 16v,Ano 2019 - 0 km,Belo Horizonte - MG,"[Bancos de couro, Freios ABS, Sensor de estaci...",antique-16040__340.png,NaN
243,R$ 203.000,SPORT,USADO,Motor 2.0 16v,Ano 2001 - 102.776 km,Belo Horizonte - MG,"[Sensor crepuscular, Sensor de chuva, Vidros e...",sport-1634268__340.png,NaN
244,R$ 474.000,IMPERIAL,USADO,Motor 1.8 16v,Ano 2011 - 101.787 km,Belo Horizonte - MG,"[Painel digital, Travas elétricas, Sensor de c...",imperial-4259819__340.png,OPORTUNIDADE


##### formatando o dataframe

In [33]:
df = pd.read_csv('./output/data/dataset.csv', sep=';')

nomes_colunas = {'value':'Valor(R$)', 'name':'Nome', 'category':'Categoria', 'motor':'Motor', 'description':'Descrição', 'location':'Local', 'items':'Itens', 'image':'Imagem', 'opportunity':'Oportunidade'}
df.rename(columns=nomes_colunas, inplace=True)

df.index = range(1,(len(df)+1))
df.index.name = 'ID'

df

,Valor(R$),Nome,Categoria,Motor,Descrição,Local,Itens,Imagem,Oportunidade
ID,,,,,,,,,
1,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"['4 X 4', 'Câmera de estacionamento', 'Control...",lamborghini-aventador-2932196__340.png,NaN
2,R$ 346.000,BMW M2,USADO,Motor 3.0 32v,Ano 2018 - 83.447 km,Belo Horizonte - MG,"['Câmera de estacionamento', 'Controle de esta...",bmw-m2-2970882__340.png,NaN
3,R$ 480.000,ALFA,USADO,Motor 1.8 16v,Ano 2004 - 19.722 km,Rio de Janeiro - RJ,"['Central multimídia', 'Bancos de couro', 'Rod...",alfa-1823056__340.png,NaN
4,R$ 133.000,PUECH,USADO,Motor Diesel V8,Ano 1992 - 34.335 km,São Paulo - SP,"['Bancos de couro', 'Freios ABS', 'Rodas de li...",puech-4055386__340.png,NaN
5,R$ 175.000,LAMBORGHINI MURCIELAGO,USADO,Motor 1.0 8v,Ano 1991 - 464 km,Belo Horizonte - MG,"['Central multimídia', 'Teto panorâmico', 'Sen...",lamborghini-murcielago-2872974__340.png,NaN
...,...,...,...,...,...,...,...,...,...
242,R$ 489.000,SUV REAR TIRE,USADO,Motor 3.0 32v,Ano 1998 - 74.292 km,São Paulo - SP,"['Câmera de estacionamento', 'Rodas de liga', ...",suv-rear-tire-2773267__340.png,NaN
243,R$ 427.000,ANTIQUE,NOVO,Motor 2.0 16v,Ano 2019 - 0 km,Belo Horizonte - MG,"['Bancos de couro', 'Freios ABS', 'Sensor de e...",antique-16040__340.png,NaN
244,R$ 203.000,SPORT,USADO,Motor 2.0 16v,Ano 2001 - 102.776 km,Belo Horizonte - MG,"['Sensor crepuscular', 'Sensor de chuva', 'Vid...",sport-1634268__340.png,NaN


In [34]:
df['Categoria'] = df['Categoria'].apply(lambda x: x.title())
df['Nome'] = df['Nome'].apply(lambda x: x.title())
df['Oportunidade'].fillna('Não', inplace=True)
df['Oportunidade'] = df['Oportunidade'].apply(lambda x: x.replace('OPORTUNIDADE', 'Sim'))
df['Valor(R$)'] = df['Valor(R$)'].apply(lambda x: int(x.replace('R$ ','').replace('.','')))
df['Ano'] = df['Descrição'].apply(lambda x: int(x.split(' - ')[0][3:]))
df['Km'] = df['Descrição'].apply(lambda x: int((x.split(' - ')[1][:-3]).replace('.','')))
df.pop('Descrição')
df = df[['Nome', 'Categoria', 'Ano', 'Km', 'Motor', 'Itens', 'Valor(R$)', 'Local', 'Oportunidade', 'Imagem']]

df.to_csv('./output/data/dataset_formatado.csv', sep=';', index=False, encoding='utf-8-sig')
df

,Nome,Categoria,Ano,Km,Motor,Itens,Valor(R$),Local,Oportunidade,Imagem
ID,,,,,,,,,,
1,Lamborghini Aventador,Usado,1993,55286,Motor 1.8 16v,"['4 X 4', 'Câmera de estacionamento', 'Control...",338000,Belo Horizonte - MG,Não,lamborghini-aventador-2932196__340.png
2,Bmw M2,Usado,2018,83447,Motor 3.0 32v,"['Câmera de estacionamento', 'Controle de esta...",346000,Belo Horizonte - MG,Não,bmw-m2-2970882__340.png
3,Alfa,Usado,2004,19722,Motor 1.8 16v,"['Central multimídia', 'Bancos de couro', 'Rod...",480000,Rio de Janeiro - RJ,Não,alfa-1823056__340.png
4,Puech,Usado,1992,34335,Motor Diesel V8,"['Bancos de couro', 'Freios ABS', 'Rodas de li...",133000,São Paulo - SP,Não,puech-4055386__340.png
5,Lamborghini Murcielago,Usado,1991,464,Motor 1.0 8v,"['Central multimídia', 'Teto panorâmico', 'Sen...",175000,Belo Horizonte - MG,Não,lamborghini-murcielago-2872974__340.png
...,...,...,...,...,...,...,...,...,...,...
242,Suv Rear Tire,Usado,1998,74292,Motor 3.0 32v,"['Câmera de estacionamento', 'Rodas de liga', ...",489000,São Paulo - SP,Não,suv-rear-tire-2773267__340.png
243,Antique,Novo,2019,0,Motor 2.0 16v,"['Bancos de couro', 'Freios ABS', 'Sensor de e...",427000,Belo Horizonte - MG,Não,antique-16040__340.png
244,Sport,Usado,2001,102776,Motor 2.0 16v,"['Sensor crepuscular', 'Sensor de chuva', 'Vid...",203000,Belo Horizonte - MG,Não,sport-1634268__340.png
